In [1]:
import zmq
import sys
import threading
import time
from random import randint, random

__author__ = "Felipe Cruz <felipecruz@loogica.net>"
__license__ = "MIT/X11"

def tprint(msg):
    """like print, but won't get newlines confused with multiple threads"""
    sys.stdout.write(msg + '\n')
    sys.stdout.flush()

class ClientTask(threading.Thread):
    """ClientTask"""
    def __init__(self, id):
        self.id = id
        threading.Thread.__init__ (self)

    def run(self):
        context = zmq.Context()
        socket = context.socket(zmq.DEALER)
        identity = u'worker-%d' % self.id
        socket.identity = identity.encode('ascii')
        socket.connect('tcp://localhost:5570')
        print('Client %s started' % (identity))
        poll = zmq.Poller()
        poll.register(socket, zmq.POLLIN)
        reqs = 0
        while True:
            reqs = reqs + 1
            print('Req #%d sent..' % (reqs))
            socket.send_string(u'request #%d' % (reqs))
            for i in range(5):
                sockets = dict(poll.poll(1000))
                if socket in sockets:
                    msg = socket.recv()
                    tprint('Client %s received: %s' % (identity, msg))

        socket.close()
        context.term()

class ServerTask(threading.Thread):
    """ServerTask"""
    def __init__(self):
        threading.Thread.__init__ (self)

    def run(self):
        context = zmq.Context()
        frontend = context.socket(zmq.ROUTER)
        frontend.bind('tcp://*:5570')

        backend = context.socket(zmq.DEALER)
        backend.bind('inproc://backend')

        workers = []
        for i in range(5):
            worker = ServerWorker(context)
            worker.start()
            workers.append(worker)

        zmq.proxy(frontend, backend)

        frontend.close()
        backend.close()
        context.term()

class ServerWorker(threading.Thread):
    """ServerWorker"""
    def __init__(self, context):
        threading.Thread.__init__ (self)
        self.context = context

    def run(self):
        worker = self.context.socket(zmq.DEALER)
        worker.connect('inproc://backend')
        tprint('Worker started')
        while True:
            ident, msg = worker.recv_multipart()
            tprint('Worker received %s from %s' % (msg, ident))
            replies = randint(0,4)
            for i in range(replies):
                time.sleep(1. / (randint(1,10)))
                worker.send_multipart([ident, msg])

        worker.close()

def main():
    """main function"""
    server = ServerTask()
    server.start()
    for i in range(3):
        client = ClientTask(i)
        client.start()

    server.join()

if __name__ == "__main__":
    main()

Worker started
Client worker-0 startedWorker started
Client worker-1 startedWorker started
Client worker-2 started


Worker started
Worker started
Req #1 sent..Req #1 sent..Req #1 sent..Worker received b'request #1' from b'worker-0'

Worker received b'request #1' from b'worker-1'

Worker received b'request #1' from b'worker-2'

Client worker-2 received: b'request #1'
Client worker-1 received: b'request #1'
Client worker-0 received: b'request #1'
Req #2 sent..
Worker received b'request #2' from b'worker-0'
Req #2 sent..
Worker received b'request #2' from b'worker-2'
Req #2 sent..
Worker received b'request #2' from b'worker-1'
Client worker-0 received: b'request #2'
Client worker-2 received: b'request #2'
Client worker-1 received: b'request #2'
Client worker-0 received: b'request #2'
Client worker-0 received: b'request #2'
Client worker-0 received: b'request #2'
Req #3 sent..
Worker received b'request #3' from b'worker-0'
Client worker-0 received: b'request #3'
Client worker-0 received: 

KeyboardInterrupt: 

Client worker-0 received: b'request #4'
Client worker-0 received: b'request #4'
Req #4 sent..
Worker received b'request #4' from b'worker-2'
Req #4 sent..
Worker received b'request #4' from b'worker-1'
Client worker-2 received: b'request #4'
Client worker-2 received: b'request #4'
Client worker-1 received: b'request #4'
Client worker-2 received: b'request #4'
Client worker-1 received: b'request #4'
Client worker-1 received: b'request #4'
Client worker-2 received: b'request #4'
Client worker-1 received: b'request #4'
Req #5 sent..
Worker received b'request #5' from b'worker-0'
Req #5 sent..
Worker received b'request #5' from b'worker-2'
Req #5 sent..
Worker received b'request #5' from b'worker-1'
Client worker-1 received: b'request #5'
Req #6 sent..
Worker received b'request #6' from b'worker-1'
Client worker-1 received: b'request #6'
Req #6 sent..
Worker received b'request #6' from b'worker-0'
Client worker-0 received: b'request #6'
Client worker-1 received: b'request #6'
Req #6 sent..

Client worker-2 received: b'request #23'
Req #22 sent..
Client worker-2 received: b'request #23'
Worker received b'request #22' from b'worker-1'
Client worker-2 received: b'request #23'
Client worker-1 received: b'request #22'
Client worker-1 received: b'request #22'
Req #20 sent..
Worker received b'request #20' from b'worker-0'
Req #24 sent..
Worker received b'request #24' from b'worker-2'
Client worker-2 received: b'request #24'
Client worker-2 received: b'request #24'
Req #23 sent..
Worker received b'request #23' from b'worker-1'
Client worker-1 received: b'request #23'
Client worker-1 received: b'request #23'
Client worker-1 received: b'request #23'
Client worker-1 received: b'request #23'
Req #25 sent..
Worker received b'request #25' from b'worker-2'
Req #24 sent..
Worker received b'request #24' from b'worker-1'
Req #21 sent..
Worker received b'request #21' from b'worker-0'
Client worker-2 received: b'request #25'
Client worker-1 received: b'request #24'
Client worker-2 received: 

Client worker-0 received: b'request #37'
Client worker-0 received: b'request #37'
Client worker-0 received: b'request #37'
Req #42 sent..
Worker received b'request #42' from b'worker-2'
Client worker-2 received: b'request #42'
Req #43 sent..
Worker received b'request #43' from b'worker-1'
Req #38 sent..
Worker received b'request #38' from b'worker-0'
Client worker-0 received: b'request #38'
Client worker-1 received: b'request #43'
Client worker-0 received: b'request #38'
Client worker-1 received: b'request #43'
Req #43 sent..
Worker received b'request #43' from b'worker-2'
Req #39 sent..
Worker received b'request #39' from b'worker-0'
Req #44 sent..
Worker received b'request #44' from b'worker-1'
Client worker-0 received: b'request #39'
Client worker-1 received: b'request #44'
Client worker-0 received: b'request #39'
Client worker-1 received: b'request #44'
Client worker-0 received: b'request #39'
Client worker-1 received: b'request #44'
Req #40 sent..
Worker received b'request #40' fr

...

Req #203 sent..
Worker received b'request #203' from b'worker-0'
Client worker-2 received: b'request #211'
Client worker-0 received: b'request #203'
Req #207 sent..
Worker received b'request #207' from b'worker-1'
Client worker-1 received: b'request #207'
Req #212 sent..
Worker received b'request #212' from b'worker-2'
Client worker-2 received: b'request #212'
Req #204 sent..
Worker received b'request #204' from b'worker-0'
Client worker-2 received: b'request #212'
Client worker-2 received: b'request #212'
Req #208 sent..
Worker received b'request #208' from b'worker-1'
Client worker-2 received: b'request #212'
Req #213 sent..
Worker received b'request #213' from b'worker-2'
Client worker-1 received: b'request #208'
Client worker-1 received: b'request #208'
Client worker-0 received: b'request #204'
Client worker-1 received: b'request #208'
Client worker-0 received: b'request #204'
Client worker-0 received: b'request #204'
Client worker-1 received: b'request #208'
Req #209 sent..Worker 

Client worker-0 received: b'request #223'
Client worker-2 received: b'request #230'
Client worker-0 received: b'request #223'
Client worker-2 received: b'request #230'
Client worker-2 received: b'request #230'
Client worker-2 received: b'request #230'
Req #226 sent..
Worker received b'request #226' from b'worker-1'
Req #224 sent..
Worker received b'request #224' from b'worker-0'
Client worker-1 received: b'request #226'
Client worker-1 received: b'request #226'
Client worker-0 received: b'request #224'
Req #231 sent..
Worker received b'request #231' from b'worker-2'
Client worker-0 received: b'request #224'
Client worker-0 received: b'request #224'
Client worker-2 received: b'request #231'
Client worker-0 received: b'request #224'
Req #225 sent..
Worker received b'request #225' from b'worker-0'
Client worker-0 received: b'request #225'
Client worker-0 received: b'request #225'
Client worker-0 received: b'request #225'
Req #227 sent..
Worker received b'request #227' from b'worker-1'
Cli

Client worker-0 received: b'request #240'
Client worker-0 received: b'request #240'
Client worker-0 received: b'request #240'
Req #247 sent..
Worker received b'request #247' from b'worker-2'
Client worker-2 received: b'request #247'
Req #245 sent..
Worker received b'request #245' from b'worker-1'
Req #241 sent..
Worker received b'request #241' from b'worker-0'
Client worker-1 received: b'request #245'
Client worker-0 received: b'request #241'
Client worker-0 received: b'request #241'
Client worker-1 received: b'request #245'
Client worker-0 received: b'request #241'
Client worker-1 received: b'request #245'
Client worker-0 received: b'request #241'
Req #242 sent..
Worker received b'request #242' from b'worker-0'
Req #246 sent..
Worker received b'request #246' from b'worker-1'
Client worker-0 received: b'request #242'
Client worker-0 received: b'request #242'
Client worker-0 received: b'request #242'
Client worker-1 received: b'request #246'
Client worker-1 received: b'request #246'
Cli

Req #259 sent..
Worker received b'request #259' from b'worker-0'
Client worker-2 received: b'request #264'
Client worker-0 received: b'request #259'
Client worker-0 received: b'request #259'
Client worker-0 received: b'request #259'
Req #262 sent..
Worker received b'request #262' from b'worker-1'
Req #265 sent..
Worker received b'request #265' from b'worker-2'
Req #260 sent..
Worker received b'request #260' from b'worker-0'
Client worker-0 received: b'request #260'
Client worker-0 received: b'request #260'
Client worker-0 received: b'request #260'
Req #261 sent..
Worker received b'request #261' from b'worker-0'
Client worker-0 received: b'request #261'
Client worker-0 received: b'request #261'
Client worker-0 received: b'request #261'
Req #263 sent..
Worker received b'request #263' from b'worker-1'
Req #266 sent..
Worker received b'request #266' from b'worker-2'
Client worker-1 received: b'request #263'
Client worker-0 received: b'request #261'
Req #262 sent..
Worker received b'request

Client worker-0 received: b'request #278'
Client worker-1 received: b'request #279'
Req #281 sent..
Worker received b'request #281' from b'worker-2'
Req #279 sent..
Worker received b'request #279' from b'worker-0'
Client worker-0 received: b'request #279'
Client worker-0 received: b'request #279'
